# HDS5210-2022 Midterm

In the midterm, you're going to use all the programming and data management skills you've developed so far to build a risk calculator that pretends to be integrated with a clinical registry.  You'll compute the PRIEST COVID-19 Clinical Severity Score for a series of patients and, based on their risk of an adverse outcome, query a REST web service to find a hospital to transfer them to. The end result of your work will be a list of instructions on where each patient should be discharged given their risk and various characteristics of the patient.

Each step in the midterm will build up to form your complete solution.

**Make sure you write good docstrings and doctests along the way!!**

**The midterm is due at 11:59 PM CST on Monday, October 23rd.**

---

## Step 1: Calculate PRIEST Clinical Severity Score

This scoring algorithm can be found [here on the MDCalc website](https://www.mdcalc.com/priest-covid-19-clinical-severity-score#evidence).  

1. You will need to write a function called **priest()** with the following input parameters.  
 * Sex (Gender assigned at birth)
 * Age in years
 * Respiratory rate in breaths per minute
 * Oxygen saturation as a percent between 0 and 1
 * Heart rate in beats per minute
 * Systolic BP in mmHg
 * Temperature in degrees C
 * Alertness as a string description
 * Inspired Oxygen as as string description
 * Performance Status as a string description
2. The function will need to follow the algorithm provided on the MDCalc website to compute a risk percentage that should be returned as a numeric value between 0 and 1.
3. Be sure to use docstring documentation and at least three built-in docstring test cases.
4. Assume that the input values that are strings could be any combination of upper or lower case. For example: 'male', 'Male', 'MALE', 'MalE' should all be interpretted by your code as male.
5. If any of the inputs are invalid (for example a sex value that is not recognizable as male or female) your code should return None.

NOTES:
1. In the final step there is a table that translates from **PRIEST Score** to **30-day probability of an outcome** but the last two probabilities are shown as ranges (59-88% and >99%).  Our code needs to output a single number, however. For our code, use the following rule:
 * If PRIEST score is between 17 and 25, the probability you return should be 0.59
 * If PRIEST score is greater than or equal to 26, the probability you return should be 0.99


In [30]:
def priest(sex, age, respiratory_rate, oxygen_saturation, heart_rate, systolic_bp, temperature, alertness, inspired_oxygen, performance_status):
    """
    Calculates the PRIEST Score and returns the 30-day probability of an outcome.

    Parameters:
    - sex (str): Gender assigned at birth (either 'male' or 'female')
    - age (int): Age in years
    - respiratory_rate (int): Respiratory rate in breaths per minute
    - oxygen_saturation (float): Oxygen saturation as a percent between 0 and 1
    - heart_rate (int): Heart rate in beats per minute
    - systolic_bp (int): Systolic BP in mmHg
    - temperature (float): Temperature in degrees C
    - alertness (str): Alertness as a string description ('alert' or 'confused or not alert')
    - inspired_oxygen (str): Inspired Oxygen as a string description ('air' or 'supplemental oxygen')
    - performance_status (str): Performance Status as a string description

    Returns:
    - float: 30-day probability of an outcome between 0 and 1. Returns None if any input is invalid.

    Examples:
    >>> priest('male', 75, 23, 0.91, 100, 95, 39.2, 'alert', 'supplemental oxygen', 'limited strenuous activity')
    0.59
    >>> priest('male', 85, 30, 0.9, 140, 85, 39.5, 'confused or not alert', 'supplemental oxygen', 'bed/chair bound, no self-care')
    0.99
    """

    # Initialize the score
    score = 0

    # Sex
    sex = sex.lower()
    if sex == 'male':
        score += 1
    elif sex != 'female':
        return None

    # Age
    if 16 <= age <= 49:
        score += 0
    elif 50 <= age <= 65:
        score += 2
    elif 66 <= age <= 80:
        score += 3
    elif age > 80:
        score += 4
    else:
        return None

    # Respiratory rate
    if respiratory_rate < 9:
        score += 3
    elif 9 <= respiratory_rate <= 11:
        score += 1
    elif 12 <= respiratory_rate <= 20:
        score += 0
    elif 21 <= respiratory_rate <= 24:
        score += 2
    elif respiratory_rate > 24:
        score += 3
    else:
        return None

    # Oxygen saturation
    if oxygen_saturation > 0.95:
        score += 0
    elif 0.94 <= oxygen_saturation <= 0.95:
        score += 1
    elif 0.92 <= oxygen_saturation <= 0.93:
        score += 2
    elif oxygen_saturation < 0.92:
        score += 3
    else:
        return None

    # Heart rate
    if heart_rate < 41:
        score += 3
    elif 41 <= heart_rate <= 50:
        score += 1
    elif 51 <= heart_rate <= 90:
        score += 0
    elif 91 <= heart_rate <= 110:
        score += 1
    elif 111 <= heart_rate <= 130:
        score += 2
    elif heart_rate > 130:
        score += 3
    else:
        return None

    # Systolic BP
    if systolic_bp < 91:
        score += 3
    elif 91 <= systolic_bp <= 100:
        score += 2
    elif 101 <= systolic_bp <= 110:
        score += 1
    elif 111 <= systolic_bp <= 219:
        score += 0
    elif systolic_bp > 219:
        score += 3
    else:
        return None

    # Temperature
    if temperature < 35.1:
        score += 3
    elif 35.1 <= temperature <= 36.0:
        score += 1
    elif 36.1 <= temperature <= 38.0:
        score += 0
    elif 38.1 <= temperature <= 39.0:
        score += 1
    elif temperature > 39.0:
        score += 2
    else:
        return None

    # Alertness
    alertness = alertness.lower()
    if alertness == 'alert':
        score += 0
    elif 'confused' in alertness or 'not alert' in alertness:
        score += 3
    else:
        return None

    # Inspired oxygen
    inspired_oxygen = inspired_oxygen.lower()
    if inspired_oxygen == 'air':
        score += 0
    elif 'supplemental oxygen' in inspired_oxygen:
        score += 2
    else:
        return None

    # Performance status
    performance_status = performance_status.lower()
    if 'unrestricted normal activity' in performance_status:
        score += 0
    elif 'limited strenuous activity' in performance_status:
        score += 1
    elif 'limited activity, can self-care' in performance_status:
        score += 2
    elif 'limited self-care' in performance_status:
        score += 3
    elif 'bed/chair bound' in performance_status:
        score += 4
    else:
        return None

    # Convert score to 30-day probability of an outcome
    if score <= 16:
        return 0.0
    elif 17 <= score <= 25:
        return 0.59
    elif score >= 26:
        return 0.99

In [31]:
import doctest
doctest.run_docstring_examples(priest, globals(),verbose=True)

Finding tests in NoName
Trying:
    priest('male', 75, 23, 0.91, 100, 95, 39.2, 'alert', 'supplemental oxygen', 'limited strenuous activity')
Expecting:
    0.59
ok
Trying:
    priest('male', 85, 30, 0.9, 140, 85, 39.5, 'confused or not alert', 'supplemental oxygen', 'bed/chair bound, no self-care')
Expecting:
    0.99
ok


## Part 2: Find a hospital

The next thing we have to do is figure out where to send this particular patient.  The guidelines on where to send a patient are based on their age (pediatric, adult, geriatric), sex, and risk percentage.  Luckily, you don't have to implement these rules. I already have. All you have to do is use a REST web service that I've created for you.

You'll want to use Python to make a call to my REST web service similar to the example URL below. The first part of the URL will be the same for everyone and every request that you make. What you will need to modify for each of your requests is the information after the question mark.

```
https://oumdj6oci2.execute-api.us-east-1.amazonaws.com/prd/?age=40&sex=male&risk_pct=0.1
```

The example above asks my web service where a 40-year old male with a risk of 10% should go.  What the web service will return back is a JSON string containing the information you need.  That JSON will look like this:

```json
{
  "age": "40",
  "sex": "male",
  "risk": "0.1",
  "hospital": "Southwest Hospital and Medical Center"
}
```

My function is not smart enough to understand `'MALE'` is the same as `'male'`.  You have to send it exactly `'male'` or `'female'`

1. Your job is to write a function called **find_hospital()** that takes age, sex, and risk as parameters.
2. Your function should call this REST web service using the `requests` module
3. Then your function will need to interpret the JSON it gets and return just the name of the hospital
4. If anything fails, return None
5. Include a good docstring with at least three test cases.


In [32]:
import requests

def find_hospital(age, sex, risk):
    """
    Finds the appropriate hospital for a patient based on age, sex, and risk percentage by querying a REST web service.

    Parameters:
    - age (int): Age of the patient
    - sex (str): Gender of the patient (either 'male' or 'female')
    - risk (float): Risk percentage of the patient (between 0 and 1)

    Returns:
    - str: Name of the recommended hospital. Returns None if there's an error in fetching or parsing the data.

    Examples:
    >>> find_hospital(40, 'male', 0.1)
    'Southwest Hospital and Medical Center'
    >>> find_hospital(50, 'female', 0.05)
    'Select Specialty Hospital - Northeast Atlanta'
    >>> find_hospital(30, 'male', 0.2)
    'Southwest Hospital and Medical Center'
    """

    base_url = "https://oumdj6oci2.execute-api.us-east-1.amazonaws.com/prd/"
    query_params = {
        "age": age,
        "sex": sex.lower(),
        "risk_pct": risk
    }

    try:
        response = requests.get(base_url, params=query_params)
        if response.status_code == 200:
            data = response.json()
            return data.get('hospital', None)
        else:
            return None
    except Exception as e:
        print(f"Error while fetching hospital data: {e}")
        return None
print(find_hospital(40, 'male', 0.1))
print(find_hospital(50, 'female', 0.05))
print(find_hospital(30, 'male', 0.2))

Southwest Hospital and Medical Center
Select Specialty Hospital - Northeast Atlanta
Southwest Hospital and Medical Center


In [33]:
import doctest
doctest.run_docstring_examples(find_hospital, globals(),verbose=True)

Finding tests in NoName
Trying:
    find_hospital(40, 'male', 0.1)
Expecting:
    'Southwest Hospital and Medical Center'
ok
Trying:
    find_hospital(50, 'female', 0.05)
Expecting:
    'Select Specialty Hospital - Northeast Atlanta'
ok
Trying:
    find_hospital(30, 'male', 0.2)
Expecting:
    'Southwest Hospital and Medical Center'
ok


## Part 3: Get the address for that hospital from a JSON file

Great! Now we have code to tell us which hospital to send someone to... but we don't know where that hospital is. The next function we need to create is one that looks up the address of that hospital.  All of these hospitals are in Atlanta, Georgia.  We're going to use the list from this webpage to lookup the address for that hospital, based on its name.  https://www.officialusa.com/stateguides/health/hospitals/georgia.html

Because we skipped the section about Beautiful Soup and working with HTML, I've converted this information into a JSON document for you.  It's available for you here.  Your code should retrieve this file using the `requests` module.

`https://drive.google.com/uc?export=download&id=1fIFD-NkcdiMu941N4GjyMDWxiKsFJBw-`

1. You need to create a function called **get_address()** that takes hospital name as a parameter and searches the data from this JSON file for the hospital you want to find.
2. Your code will have to load the JSON and return the correct hospital based on name.
3. If the hospital name isn't found, the function should return None.
4. Be sure to use good docstring documentation and includes at least 3 test cases.

In [34]:
def get_address(hospital_name):
    """
    Retrieves the address of a hospital based on its name from a JSON data source.

    Parameters:
    - hospital_name (str): Name of the hospital

    Returns:
    - str: Address of the hospital. Returns None if the hospital name isn't found.

    Examples:
    >>> get_address('COOK MEDICAL CENTER')
    '706 NORTH PARRISH AVENUE'

    >>> get_address('PHOEBE PUTNEY MEMORIAL HOSPITAL')
    '417 THIRD AVENUE'

    >>> get_address('Non-existent Hospital')
    None
    """

    try:

        response = requests.get(url)
        data = response.json()
        for hospital in data:
            if hospital_name.upper() in hospital:
                return data[hospital]["ADDRESS"]
        return None
    except KeyError:
        return None
    except Exception as e:
        return None
get_address('EMORY-ADVENTIST HOSPITAL')


## Part 4: Run the risk calculator on a population

At the link below, there is a file called `people.psv`.  It is a pipe-delimited (`|`) file with columns that match the inputs for the PRIEST calculation above.  Your code should use the `requests` module to retrieve the file from this URL.

`https://drive.google.com/uc?export=download&id=1fLxJN9YGUqmqExrilxSS8furwUER5HHh`


In addition, the file has a patient identifier in the first column.

1. Write a function called **process_people()** that takes the file location above as its only parameter. Your Python program should use your code above to process all of these rows, determine the hospital and address, and return a list whose items are a dictionary like this: `{ patient_number: [sex, age, breath, o2sat, heart, systolic, temp, alertness, inspired, status, hospital, address]}`.  Look at the file in Part 5 for what the output looks like.
2. Be sure to use good docstrings, but you don't need any tests in your doc strings.  I've provided those for you withe file in Part 5.


**NOTE** that when running your code for all the 100 records in the `people.psv` file, it may take a few minutes to complete.  You're making multiple calls to the internet for each record, so that can take a little while.


In [35]:
import requests

def process_people(url):
    """
    Processes a pipe-delimited file of patient data to calculate the PRIEST risk score,
    determine the recommended hospital, and fetch the hospital address.

    Parameters:
    - url (str): The URL of the pipe-delimited file.

    Returns:
    - list: A list of dictionaries, each representing a patient's data, including the
            calculated PRIEST risk score, recommended hospital, and hospital address.
    """
    try:
        response = requests.get(url)
        response.raise_for_status()
    except requests.RequestException:
        return None

    lines = response.text.strip().split('\n')
    headers = lines[0].split('|')

    results = []

    expected_number_of_columns = len(headers)
    for line in lines[1:]:
        if not line:
            continue

        data_points = line.split('|')

        if len(data_points) < expected_number_of_columns:
            continue

        patient_number = data_points[0]
        sex = data_points[1]
        age = int(data_points[2])
        breath = int(data_points[3])
        o2sat = float(data_points[4])
        heart = int(data_points[5])
        systolic = int(data_points[6])
        temp = float(data_points[7])
        alertness = data_points[8]
        inspired = data_points[9]
        status = data_points[10]

        risk_score = priest(sex, age, breath, o2sat, heart, systolic, temp, alertness, inspired, status)
        hospital = find_hospital(age, sex, risk_score)

        # Add the data to results
        results.append({
            patient_number: [sex, age, breath, o2sat, heart, systolic, temp, alertness, inspired, status, hospital]
        })

    return results

url = "https://hds5210-data.s3.amazonaws.com/people.psv"
result = process_people(url)
print(result)

[{'E9559': ['FEMALE', 40, 24, 0.96, 105, 115, 34.9, 'ALERT', 'AIR', 'unrestricted normal activity', 'Select Specialty Hospital - Northeast Atlanta']}, {'E9385': ['Female', 51, 19, 0.99, 90, 91, 38.1, 'ALERT', 'supplemental oxygen', 'unrestricted normal activity', 'Select Specialty Hospital - Northeast Atlanta']}, {'E3067': ['female', 40, 29, 0.96, 105, 95, 38.1, 'ALERT', 'air', 'unrestricted normal activity', 'Select Specialty Hospital - Northeast Atlanta']}, {'E9422': ['FEMALE', 66, 19, 0.96, 135, 115, 38.1, 'ALERT', 'AIR', 'bed/chair bound, no self-care', 'Wesley Woods Geriatric Hospital']}, {'E8661': ['MALE', 18, 22, 0.95, 105, 115, 36.3, 'Confused or Not Alert', 'air', 'unrestricted normal activity', 'Southwest Hospital and Medical Center']}, {'E6235': ['Male', 16, 23, 0.96, 135, 115, 39.2, 'ALERT', 'supplemental oxygen', 'unrestricted normal activity', 'Childrens Healthcare of Atlanta at Scottish Rite']}, {'E4451': ['MALE', 40, 8, 0.96, 135, 115, 38.1, 'ALERT', 'AIR', 'unrestricte

## Part 5: Checking your final results

The final step is to check your results.  You should be able to compare your results to the output in `people_results.json` at the link below.  Write some code to check your results.  This does not need to be a function.

`https://drive.google.com/uc?export=download&id=1gx1SSC20mO5XL6uYD0mdcM_cL91fcIW5`


In [36]:
import requests

def compare():
    fileURL = "https://hds5210-data.s3.amazonaws.com/people_results.json"
    response = requests.get(fileURL)

    if response.status_code != 200:
        print("Failed to retrieve the data.")
        return None

    data_from_file = response.json()
    result_from_function = process_people("https://hds5210-data.s3.amazonaws.com/people.psv")
    return data_from_file == result_from_function
result = compare()
print(result)


False


---

## Check your work above

If you didn't get them all correct, take a few minutes to think through those that aren't correct.


## Submitting Your Work

Submit your work as usual into a folder named `midterm`

---